# Multiprocessing: Interprocess communication
_COSC 208, Introduction to Computer Systems, 2023-11-10_

## Announcements
* Project 3 due Monday @ 11pm

## Outline
* Interprocess communication
* Memory hierarchy

## Interprocess communication

### Signals
* Send using the `kill` system call
* Register a signal handler using the `signal` system call – must register before calling fork

* Q1: _What does the following program output assuming the process identifier (PID) of the parent process is 3995 and the PID of the child process is 3996? (Note: `getpid` gets the PID of the calling process, and `getppid` gets the PID of the calling process's parent process.)_

In [3]:
#include <signal.h>
#include <stdio.h>
#include <unistd.h>

void signal_handler(int sig) {
    if (sig == SIGUSR1) {
        printf("SIGUSR1 %d\n", getpid());
    } else if (sig == SIGUSR2) {
        printf("SIGUSR2 %d\n", getpid());
    }
}

int main() {
    if (signal(SIGUSR1, signal_handler) == SIG_ERR) {
        return 1;
    }
    if (signal(SIGUSR2, signal_handler) == SIG_ERR) {
        return 1;
    }
    
    int x = fork();
    if (x < 0) {
        return 1;
    } else if (x > 0) {
        printf("ElseIf %d\n", getpid());
        for (int i = 0; i < 1000*1000; i++); // Do work
        if (manual page killl
            < 0) {
            return 1;
        }
        wait(NULL);
    } else {
        printf("Else %d\n", getpid());
        for (int i = 0; i < 1000*1000; i++); // Do work
        if (kill(getppid(), SIGUSR2) < 0) {
            return 1;
        }
        for (int i = 0; i < 1000*1000; i++); // Do work
    } 
    return 0;
}

ElseIf 3995
Else 3996
SIGUSR1 3996
SIGUSR2 3995


### Message passing
* pipe – one-way communcation channel for two processes on the same machine
* socket - dual-way communication channel for two processes on the same or different machines
    * See COSC 465: Computer Networks

In [1]:
#include <signal.h>
#include <stdio.h>
#include <unistd.h>

int main() {
    int pipe_descriptors[2];
    if (pipe(pipe_descriptors) < 0) {
        return 1;
    }

    int result = fork();
    if (result < 0) {
        return 1;
    } else if (result == 0) {
        printf("Child\n");
        char msg[1024];
        read(pipe_descriptors[0], msg, 1024);
        printf("%s\n", msg);
    } else {
        char msg[1024] = "Hello from your parent!";
        write(pipe_descriptors[1], msg, 1024);
        wait(NULL);
        printf("Parent\n");
    }
    return 0;
}

Child
Hello from your parent!
Parent


<div style="page-break-after:always;"></div>

## Memory hierarchy

* Registers (special purpose circuits)
* Caches (SRAM)
* Main Memory (DRAM)
* Solid State Drive (SSD)
* Hard (spinning) disk drive (HDD)
* Remote Storage (e.g. cloud drive)

* Storage capacity
    * Let's assume 1 byte = 1mL
    * Registers — 30 * 8B = ~250mL = ~1 cup
    * Caches (Core i7 in MacBook Pro)
        * L1 — 32KB + 32KB = 64L = ~1 tank of gas
        * L2 — 512KB * 4 cores = 2048L = ~7 bathtubs
    * Main memory = 32GB (in MacBook Pro) = ~13 olympic swimming pools
    * SSD = 1TB (in MacBook Pro) = ~Lake Moraine

* Access latency
    * Let's consider a 1hz CPU, which means 1 cycle = 1 second
    * Registers — 1 cycle = 1 second
    * Caches — ~10 cycles = ~10 seconds
    * Main memory — ~100 cycles = ~2 minutes
    * Solid-state drive — ~1 million cycles = ~11.5 days
    * Hard disk drive — ~10 million cycles = ~115 days
    * Remote storage — ~20ms = ~2 years

* Cost
    * 2 x 16GB DRAM = ~\\$100 = \\$3.12 per GB
    * 1TB SSD = \\$80 = \\$0.08 per GB
    * 2TB HDD = \\$60 = \\$0.03 per GB
* Volatility
    * Primary storage (registers, caches, and main memory) — volatile (i.e., data is lost if power is lost)
    * Secondary storage (SSD, HDD, network storage) — non-volatile (i.e., data is preserved if power is lost)

## Data movement

* Recall: _How does data move between the CPU, main memory, and secondary storage in the von Neumann Architecture?_
* _Why does data move between registers and main memory?_ 
* _How can we move less data (i.e., perform fewer loads and stores)?_

* Recall: _How does data move between the CPU, main memory, and secondary storage in the von Neumann Architecture?_ - bus
* _Why does data move between registers and main memory?_ — not enough room in registers to store all values used by a program at runtime
* _How can we move less data (i.e., perform fewer loads and stores)?_

 * Better use of registers — Loads and stores are unnecessary when the value of a register is not changed between store and load instructions involving the same register and memory address
    * Example load which is _unnecessary_
        ```
        str x0, [sp,#4]
        ldr x0, [sp,#4] // Can eliminiate
        ```
    * Example load store which is _necessary_
        ```
        str w0, [sp,#4]
        mov w0, #0x1
        str w0, [sp]
        ldr w0, [sp,#4]
        ```
    * Better register assignments to eliminate loads (and stores)
        ```
        str w0, [sp,#4]
        mov w1, #0x1
        str w1, [sp]
        ldr w0, [sp,#4] // Can eliminiate
        ```
    * Must preserve calling conventions
        * Parameters are stored in w/x0, w/x1, ...
        * Return value is stored in w/x0
        * Caller must store register values into caller's stack frame before `bl` to callee — actually only needed if values in registers are needed by caller after `bl` and callee overwrites the values in those registers
* Leverage locality
    * Add additional memory to the CPU — i.e., a cache
    * Optimize code to improve locality